In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

### Load data

In [2]:
data = loadmat('data/MINST')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
print(data['X'].shape, data['y'].shape)

(5000, 400) (5000, 1)


### Encode Label

In [4]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(data['y'])
y_onehot

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

### Activation Function

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

### Forward Propagation

In [6]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    ones = np.ones([m, 1])
    # Input Layer
    a1 = np.concatenate((ones, X), axis=1)
    # Hidden Layer
    z2 =a1*theta1.T
    a2 =sigmoid(z2)
    a2 = np.concatenate((ones, a2), axis=1)
    # Output Layer
    z3 = a2*theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

### Cost Function

In [7]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
        
    J = J / m
    # cost function with regularization
    J += (float(learning_rate) / (2*m) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:],2))))
    
    return J
    

### Initialize

In [8]:
# initial setup
input_size = 400
hidden_size = 25
num_labels = 10
learning_rate = 1
# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.2

m = data['X'].shape[0]
X = np.matrix(data['X'])
y = np.matrix(data['y'])
# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

print("X")
print(X.shape)

print("\na1")
print(a1.shape)
print(a1)

print("\ntheta1")
print(theta1.shape)
# print(theta1)

print("\nz2")
print(z2.shape)
print(z2)

print("\na2")
print(a2.shape)
print(a2)

print("\ntheta2")
print(theta2.shape)
# print(theta2)

print("\nz3")
print(z3.shape)
print(z3)

print("\nh")
print(h.shape)
print(h)

X
(5000, 400)

a1
(5000, 401)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]

theta1
(25, 401)

z2
(5000, 25)
[[-0.44766294  0.02597435  0.48970539 ...  0.76010568  0.7225221
   0.1789523 ]
 [-0.13707683  0.06557284  0.37509521 ...  0.78775169  0.7633191
   0.20393546]
 [ 0.01140966 -0.24262989  0.46784691 ...  0.32154635  0.4173466
  -0.04772692]
 ...
 [ 0.26284356  0.25281572  0.15024436 ...  0.13769077 -0.16613746
   0.01632722]
 [ 0.07724789 -0.09249144  0.16680036 ... -0.0129061   0.40811216
  -0.1087338 ]
 [-0.05870841  0.19572541  0.47346771 ...  0.01144183  0.33825102
  -0.24973437]]

a2
(5000, 26)
[[1.         0.38991657 0.50649322 ... 0.68137668 0.67316216 0.54461906]
 [1.         0.46578435 0.51638734 ... 0.68734837 0.68207391 0.5508079 ]
 [1.         0.50285238 0.43963836 ... 0.57970106 0.60284814 0.48807053]
 ...
 [1.         0.56533517 0.56286942 ... 0.53436841 0.4

In [9]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

6.624363089452915

In [10]:
loss = h - y
loss.shape

(5000, 10)

### Sigmoid Gradient

In [11]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))    

### Backpropagation

In [12]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
 

    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    #forward
    J = 0
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
        
    J = J / m
    J += (float(learning_rate) / (2*m) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:],2))))

    # back
    loss3 = h - y

    gradient2 = np.matmul(np.transpose(loss3), a2)
    gradient2 = gradient2 / m
    gradient2[:,1:] += (float((learning_rate)) / m) * theta2[:,1:] 


    
    loss2 = np.array(np.matmul(loss3, theta2[:,1:])) * np.array(sigmoid_gradient(z2))
    gradient1 = np.matmul(np.transpose(loss2), a1)
    gradient1 = gradient1 / m
    gradient1[:,1:] += ((float(learning_rate)) / m) * theta1[:,1:]
    
    grad = np.array(gradient1.reshape(gradient1.shape[0] * gradient1.shape[1],))
    grad = np.append(grad, np.array(gradient2.reshape(gradient2.shape[0] * gradient2.shape[1])))
    #print("loss", J)
    #print("g:", grad.shape)
    return J, grad

### Predict

In [13]:
from scipy.optimize import minimize
# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate), method='TNC', jac=True, options={'maxiter': 250})
print("fmin: ", fmin)

X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)

correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

fmin:       fun: 0.32941428067218
     jac: array([ 1.97780041e-05, -2.46857050e-07, -1.06973442e-07, ...,
       -2.37777099e-04, -1.46922064e-04, -2.07521785e-04])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 22
  status: 3
 success: False
       x: array([ 1.32290393e+00, -1.23428525e-03, -5.34867212e-04, ...,
        1.21070075e+00,  2.82640499e+00,  5.36469822e-01])
accuracy = 99.33999999999999%
